In [1]:
from time import sleep
from contextlib import contextmanager

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

@contextmanager
def chrome_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=chrome_options)
    try:
        yield driver
    finally:
        driver.quit()

def get_html(url: str, load_time_seconds: int = 5) -> str:
    with chrome_driver() as driver:
        driver.get(url)
        sleep(load_time_seconds)
        return driver.page_source

In [14]:
from dataclasses import dataclass
from bs4 import BeautifulSoup

URL = "https://sssb.se/soka-bostad/sok-ledigt/lediga-bostader/?pagination=0&paginationantal=0"

@dataclass(frozen=True)
class Listing:
    url: str
    apartment_type: str
    adress: str
    region: str
    floor: str
    area: str
    rent: str
    move_in_date: str
    
    
def parse_raw_listing(raw_listing: BeautifulSoup) -> Listing:
    title_element = raw_listing.find(attrs={"class": "ObjektTyp"})
    url = title_element.find(href=True).get("href", "")
    apartment_type = title_element.text
    adress = raw_listing.find(attrs={"class": "ObjektAdress"}).text
    region = raw_listing.find("dd", attrs={"class": "ObjektOmrade"}).text
    floor = raw_listing.find("dd", attrs={"class": "ObjektVaning hidden-phone"}).text
    area = raw_listing.find("dd", attrs={"class": "ObjektYta"}).text
    rent = raw_listing.find("dd", attrs={"class": "ObjektHyra"}).text.replace(u"\xa0", u" ")
    move_in_date = raw_listing.find("dd", attrs={"class": "ObjektInflytt hidden-phone"}).text
    
    return Listing(
        url=url,
        apartment_type=apartment_type,
        adress=adress,
        region=region,
        floor=floor,
        area=area,
        rent=rent,
        move_in_date=move_in_date,
    )

def get_listings() -> list[Listing]:
    html = get_html(URL)
    soup = BeautifulSoup(html)
    raw_listings = soup.find_all(attrs={"class": "Box ObjektListItem"})
    return [parse_raw_listing(raw_listing)
            for raw_listing in raw_listings]
    
get_listings()

C:\Users\martin\AppData\Local\Temp\ipykernel_14976\679622810.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=chrome_options)


[Listing(url='https://sssb.se/soka-bostad/sok-ledigt/lediga-bostader/lagenhet/?refid=4c6d35304658565673584c787a553642594875675a7739535042714c3745505951496f7a625a78612b48633d', apartment_type='1 rum & pentry', adress='David Bagares Gata 6 / 1407', region='Vätan', floor='4 ', area='37 m²', rent='7 091 kr', move_in_date='2023-11-16'),
 Listing(url='https://sssb.se/soka-bostad/sok-ledigt/lediga-bostader/lagenhet/?refid=7543574d2b7066693264777979706a6f6c44413944347541472b657257577638794775634a5370314c5a493d', apartment_type='2 rum & kök', adress='Studentbacken 21 / 1117', region='Jerum', floor='1 ', area='41 m²', rent='6 615 kr', move_in_date='2023-11-16'),
 Listing(url='https://sssb.se/soka-bostad/sok-ledigt/lediga-bostader/lagenhet/?refid=776835386a31324a59554c6551744a465359697261746d443074504b797962795a51417934684b426e64733d', apartment_type='Rum i korridor', adress='Studentbacken 21 / 1511', region='Jerum', floor='5 ', area='17 m²', rent='4 142 kr', move_in_date='2023-12-01'),
 Listing(